
# Introduction to py-redis (Key-Value Database)

This notebook will guide you through the basic functionalities of `py-redis`, a Python library for working with Redis, a key-value store database.

We'll begin by installing and starting a Redis server directly on Google Colab.


In [1]:

# Install Redis server and py-redis
!apt-get install redis-server
!pip install redis


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjemalloc2 liblua5.1-0 liblzf1 lua-bitop lua-cjson redis-tools
Suggested packages:
  ruby-redis
The following NEW packages will be installed:
  libjemalloc2 liblua5.1-0 liblzf1 lua-bitop lua-cjson redis-server redis-tools
0 upgraded, 7 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,273 kB of archives.
After this operation, 5,725 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjemalloc2 amd64 5.2.1-4ubuntu1 [240 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 liblua5.1-0 amd64 5.1.5-8.1build4 [99.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 liblzf1 amd64 3.6-3 [7,444 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lua-bitop amd64 1.0.2-5 [6,680 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lua-c


## Starting Redis Server

Next, we will start the Redis server in the background.


In [2]:

# Start Redis server in the background
import os
os.system('redis-server --daemonize yes')

# Check if Redis server is running
os.system('redis-cli ping')


0


## Connecting to Redis

Now that the Redis server is running, let's connect to it using `py-redis`.


In [3]:
import redis

# Connecting to the Redis server running on Colab
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Test the connection
try:
    r.ping()
    print("Connected to Redis!")
except redis.ConnectionError:
    print("Failed to connect to Redis.")


Connected to Redis!


In the code above, the db=0 parameter specifies which logical database to use. Redis supports multiple databases, and these are numbered from 0 to 15 by default. When you connect to Redis, you are connected to database 0 by default unless you specify otherwise.

- db=0 means that you’re using the first database.
- You can change the database by setting the db parameter to another number, for example db=1 to use the second database, and so on up to db=15.

Each logical database in Redis is completely independent from the others, so keys stored in db=0 won’t be available in db=1, for example. It’s like having multiple separate Redis databases within a single instance.


## Inserting a Single Key-Value Pair

Let's start by inserting a single key-value pair into the Redis database.


In [4]:

# Inserting a single key-value pair
r.set('name', 'John Doe')

# Retrieving the value
name = r.get('name')
print(f"Stored name: {name.decode('utf-8')}")


Stored name: John Doe



## Batch Insert

Redis supports batch operations to insert multiple key-value pairs efficiently. We'll use the `mset` method for this.


In [ ]:

# Batch insert key-value pairs
batch_data = {
    'age': 30,
    'city': 'New York',
    'occupation': 'Engineer'
}

r.mset(batch_data)

# Retrieving the values to verify
for key in batch_data.keys():
    print(f"{key}: {r.get(key).decode('utf-8')}")


age: 30
city: New York
occupation: Engineer


# Batch insert: Using hset vs mset

Redis provides two useful commands for storing multiple fields:

	•	mset: Used to set multiple independent key-value pairs.
	•	hset: Used to set multiple fields within a single key (specifically for hashes).

1. Using mset: Storing Multiple Independent Key-Value Pairs

With mset, you can store unrelated key-value pairs in Redis. Each key is independent and holds its own value but first, let's create a new database:

In [22]:
rb = redis.StrictRedis(host='localhost', port=6379, db=2)

In [23]:
# Storing multiple independent key-value pairs
rb.mset({
    'user:1': 'John Doe',
    'user:2': 'Jane Smith',
    'user:3': 'Alice Johnson'
})

# Retrieving values
print(rb.get('user:1').decode('utf-8'))  # Output: John Doe
print(rb.get('user:2').decode('utf-8'))  # Output: Jane Smith
print(rb.get('user:3').decode('utf-8'))  # Output: Alice Johnson

John Doe
Jane Smith
Alice Johnson


Key Characteristics of mset:

	•	Each key is independent.
	•	It is used when the data stored under each key doesn’t need to be grouped.

2. Using hset: Storing Multiple Fields Within a Single Key

hmset is used for Redis hashes, where you want to store multiple fields (like a dictionary) under a single key.

In [24]:
# Storing data for a person using hset
rb.hset('person:1', 'name', 'John Doe')
rb.hset('person:1', 'age', '30')
rb.hset('person:1', 'city', 'New York')
rb.hset('person:1', 'occupation', 'Engineer')

# Retrieving all fields of the hash
person_data = rb.hgetall('person:1')
print({k.decode('utf-8'): v.decode('utf-8') for k, v in person_data.items()})

{'name': 'John Doe', 'age': '30', 'city': 'New York', 'occupation': 'Engineer'}


Alternatively, you can use a loop to insert multiple fields efficiently:

In [25]:
# Using a loop to set multiple fields
fields = {'name': 'John Doe', 'age': '30', 'city': 'New York', 'occupation': 'Engineer'}
for field, value in fields.items():
    rb.hset('person:1', field, value)

Key Characteristics of hset:

	•	All fields (like name, age, city) are stored under one key (person:1 in this case).
	•	This is useful for grouping related data (e.g., storing user profiles or product details).

Now let's get all the keys

In [26]:
# Get all keys in the Redis database
all_keys = rb.keys('*')

# Print the keys
for key in all_keys:
    print(key.decode('utf-8'))

user:3
user:2
person:1
user:1



## Deleting a Key-Value Pair

You can delete a key-value pair from the Redis database using the `delete` method.


In [27]:

# Deleting a key-value pair
r.delete('occupation')

# Trying to retrieve the deleted key
occupation = r.get('occupation')
print(f"Occupation after deletion: {occupation}")


Occupation after deletion: None



## Additional Redis Features

Redis supports many additional features, such as setting a TTL (time-to-live) for a key or incrementing numeric values.

### Setting TTL
You can set a TTL for a key, after which the key will automatically expire.


In [ ]:

# Setting a key with a TTL of 10 seconds
r.set('temp_key', 'This is temporary', ex=10)
print(f"temp_key: {r.get('temp_key').decode('utf-8')}")

# After 10 seconds, the key will expire
import time
time.sleep(10)

temp_key = r.get('temp_key')
print(f"temp_key after 10 seconds: {temp_key}")


temp_key: This is temporary
temp_key after 10 seconds: None



### Incrementing Numeric Values

Redis allows you to increment numeric values atomically, which is useful in many scenarios such as counters.


In [ ]:

# Incrementing a numeric value
r.set('counter', 0)
r.incr('counter')
print(f"Counter: {r.get('counter').decode('utf-8')}")

# Incrementing by 5
r.incrby('counter', 5)
print(f"Counter after incrementing by 5: {r.get('counter').decode('utf-8')}")


Counter: 1
Counter after incrementing by 5: 6
